In [1]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from PMTK.random.preferences_sampler import sample_preferences_from_order, sample_preferences_from_complete_order
from PMTK.utils import *
from PMTK.random.subset_samplers import sample_subsets
from PMTK.utility.additive_utility import AdditiveUtility
from PMTK.preferences import *
from PMTK.utility.utility_fitter import Utility_Fitter
from PMTK.utility.extension_solver import *
from tqdm import tqdm

In [2]:
def get_all_explicative(x,y):
    s_i = get_all_k_sets(x, len(x))
    s_j = get_all_k_sets(y, len(y))
    theta = []
    for x in s_i:
        if x not in s_j:
            theta.append(x)
    
    for x in s_j:
        if x not in s_i:
            theta.append(x)
    theta = list(set(theta))
    return theta

def equal_lists(L1, L2):
    return all([i in L1 for i in L2]) and all([i in L2 for i in L1])

def exists(e,L):
    for i in L:
        if equal_lists(i,e):
            return True
    return False

def keep_unique(L):
    k = []
    for i in L:
        if not i in k:
            k.append(i)
    return k

def keep_unique_tuples(L):
    k = []
    for i in L:
        if not exists(i,k):
            k.append(i)
    return k


def get_preferences(theta, items, subsets, preferences):
    UF = Utility_Fitter(items, theta)
    UF.set_preferences(preferences).set_model(theta).build_vars().build_preferences_cst()
    p = UF.get_robust_preferences(subsets, verbose=False)
    return p

def get_all_preferences(thetas, items, subsets, preferences):
    D = {}
    for theta in tqdm(thetas):
        p = get_preferences(theta, items, subsets, preferences)
        D[tuple(theta)] = p
    return D

def product(L):
    p = 1
    for i in L:
        p *= i
    return p

def get_all_thetas(preferences):
    explications = []
    for x,y in preferences.preferred:
        c = get_all_explicative(x,y)
        explications.append(c)
    L = [keep_unique(i) for i in itertools.product(*explications)]
    return keep_unique_tuples(L)

def preference_complexity(x,y):
    return 2**len(x) + 2**len(y) - (len(set(x).intersection(set(y))))

def prune_non_compatible_thetas(items, preferences, thetas):
    n_theta = []
    removed_n = 0
    for i in thetas:
        UF = Utility_Fitter(items, i)
        UF.set_preferences(preferences).set_model(i).build_vars().build_preferences_cst().run(UF.get_most_discriminant_utility)
        if UF.get_utility() != None:
            n_theta.append(i)
        else:
            removed_n = removed_n + 1
    return n_theta, removed_n


def run_simulation(items, preferences):
    L = get_all_thetas(preferences)
    theta_prunned, n_removed = prune_non_compatible_thetas(items, preferences, L)
    all_subsets = sorted(get_all_k_sets(items, len(items))+[EMPTY_SET])
    pref_dict = get_all_preferences(theta_prunned, items, all_subsets, preferences)
    return pref_dict, n_removed

def test_compatibility_with_original_prefs(items, preferences, thetas):
    print("Testing the difference between p_1 and preferences")
    for theta in prune_non_compatible_thetas(items, preferences, thetas):
        all_subsets = sorted(get_all_k_sets(items, len(items))+[EMPTY_SET])
        print("========")
        print("Theta:", theta)
        try:
            p_1 = get_preferences(theta, items, all_subsets, preferences)
        except:
            print("Cannot represent the preferences")
            continue
        print(preferences - p_1)
        break
    
def count_contradictions(pref_dict):
    theta_keys = list(pref_dict.keys())
    contradictions = np.zeros((len(pref_dict.keys()), len(pref_dict.keys())))
    produced_preferences = np.zeros(len(theta_keys))
    for theta_1 in pref_dict:
        p_1 = pref_dict[theta_1]
        produced_preferences[theta_keys.index(theta_1)] = len(p_1 - preferences)
        for theta_2 in pref_dict:
            if theta_1 == theta_2:
                continue
            p_2 = pref_dict[theta_2]
            c_1_2 = p_1.contradictions(p_2)
            contradictions[theta_keys.index(theta_1), theta_keys.index(theta_2)] = len(c_1_2)
    return contradictions, produced_preferences

def union_preferences(items, preferences):
    P = Preferences(items)
    for i in preferences:
        P = P + i
    return P

def intersection_preferences(items, preferences):
    P = union_preferences(items, preferences)
    for i in preferences:
        P = P.intersect(i)
    return P

def additivity(theta):
    L = []
    for s in theta:
        L.append(len(s))
    return max(L)
    
def compute_k_min(thetas):
    k = []
    for theta in thetas:
        k.append(additivity(theta))
    return min(k)

def compute_min_size(thetas):
    sizes = []
    for theta in thetas:
        sizes.append(len(theta))
    return min(sizes)

def get_theta_min(thetas):
    k = compute_k_min(thetas)
    theta_min = []
    for theta in thetas:
        if additivity(theta) <= k:
            theta_min.append(theta)
    size_min = compute_min_size(thetas)
    theta_min_2 = []
    for t in theta_min:
        if len(t) <= size_min:
            theta_min_2.append(t)
    return theta_min_2

def inclusion(S1,S2):
    for i in S1:
        if not i in S2:
            return False
    return True

def print_no_contradictions(pref_dict):
    thetas = list(pref_dict.keys())
    contradictions, produced_pref = count_contradictions(pref_dict)
    k = np.where(contradictions == 0.0)
    x_y = set([(x, y) for x,y in zip(k[0], k[1])])
    for x,y in x_y:
        print(thetas[x] ,"===" ,thetas[y])



In [3]:
def generate_tractable_prefs(n_items, n_preferences, t_min = 10000, t_max = 30000, MAX_ITER = 100):
    cpt = 0
    while(True):
        cpt = cpt + 1
        if(cpt == MAX_ITER):
            print("Reached max iter without finding")
            return None
        n_items = 4
        n_preferences = 5
        items = np.arange(n_items)
        preferences = sample_preferences_from_order(items, n_preferences, indifference_rate=0).sort_by_n_candidates()
        explications = []
        for x,y in preferences.preferred:
            c = get_all_explicative(x,y)
            explications.append(c)
        n = product([len(i) for i in explications])
        if n >= t_min and n <= t_max: 
            return preferences

In [4]:
p = generate_tractable_prefs(4, 10)

In [5]:
cpt = 0
datas = []
generated_prefs = []
n_items = 4
n_preferences = 12

for i in range(1,11):
    
    #Generate the instance
    items = np.arange(n_items)
    preferences = generate_tractable_prefs(n_items, n_preferences, t_min = 1000*i, t_max = 2000*i)
    
    #Run the simulation
    pref_dict, n_removed = run_simulation(items, preferences)
    
    #Get theta and theta_min
    thetas = list(pref_dict.keys())
    min_thetas = get_theta_min(thetas)
    min_pref_dict = {i:pref_dict[i] for i in min_thetas}
    
    #Count contradictions
    contradiction_theta, produced_theta = count_contradictions(pref_dict)
    contradiction_theta_min, produced_theta_min = count_contradictions(min_pref_dict)
    
    # Getting non contradictory models in theta and in theta min 
    non_contradictory_theta = [thetas[i] for i in np.where(contradiction_theta.sum(axis=0) == 0)[0]]
    non_contradictory_theta_min = [min_thetas[i] for i in np.where(contradiction_theta_min.sum(axis=0) == 0)[0]]
    
    print("Theta min: ", min_thetas)
    #print("Theta:", thetas)
    print("non contradictory thetas : ", non_contradictory_theta)
    print("non contradictory theta min : ", non_contradictory_theta_min)
    
    
    if not inclusion(non_contradictory_theta_min, non_contradictory_theta):
        print("Elements that are not contradictory in theta min are not in the not contradictory elements of theta")
    
    for t in min_thetas:
        if t in non_contradictory_theta and t not in non_contradictory_theta_min:
            print(f"Error, {t} is not contradictory in global but is contradictory compared to elements of theta min")
            
    ## Analyzing the insersections
    preferences_theta = list(pref_dict.values())
    preferences_theta_min = list(min_pref_dict.values())
    
    intersection_theta = intersection_preferences(items, preferences_theta) - preferences
    intersection_theta_min = intersection_preferences(items, preferences_theta_min) - preferences
    print("Size of the intersection of theta preferences: ", len(intersection_theta))
    print("Size of the intersection of theta_min preferences:", len(intersection_theta_min)) 
    

100%|██████████| 42/42 [07:38<00:00, 10.93s/it]


Theta min:  [((2,), (1,), (0,))]
non contradictory thetas :  [((2,), (1,), (0, 1), (1, 2), (0,)), ((2,), (1,), (0, 1), (1, 2), (0, 2)), ((2,), (1,), (0, 1), (1, 2), (0, 1, 2)), ((2,), (1,), (0, 1), (1, 2)), ((2,), (1,), (1, 2), (0,)), ((2,), (1,), (1, 2), (0, 2)), ((2,), (1,), (1, 2), (0, 1, 2)), ((2,), (1,), (1, 2), (0, 1, 2), (0,)), ((2,), (1,), (1, 2), (0, 1, 2), (0, 2)), ((2,), (1,), (1, 2), (0, 2), (0,)), ((2,), (1,), (1, 2), (0, 1), (0, 1, 2), (0,)), ((2,), (1,), (1, 2), (0, 1), (0, 1, 2), (0, 2)), ((2,), (1,), (1, 2), (0, 1), (0, 2), (0,)), ((2,), (1,), (1, 2), (0, 1, 2), (0, 2), (0,))]
non contradictory theta min :  [((2,), (1,), (0,))]
Elements that are not contradictory in theta min are not in the not contradictory elements of theta
Size of the intersection of theta preferences:  34
Size of the intersection of theta_min preferences: 114


100%|██████████| 55/55 [08:28<00:00,  9.25s/it]


Theta min:  [((0,), (1,)), ((0,), (2,))]
non contradictory thetas :  [((0,), (1,), (0, 1), (2,)), ((0,), (1,), (0, 1), (2,), (0, 2)), ((0,), (1,), (0, 1), (2,), (0, 1, 2)), ((0,), (1,), (0, 1), (1, 2)), ((0,), (1,), (0, 1), (1, 2), (0, 2)), ((0,), (1,), (0, 1), (1, 2), (0, 1, 2)), ((0,), (1,), (2,)), ((0,), (1,), (2,), (0, 2)), ((0,), (1,), (2,), (0, 1, 2)), ((0,), (1,), (2,), (1, 2)), ((0,), (1,), (2,), (1, 2), (0, 2)), ((0,), (1,), (2,), (1, 2), (0, 1, 2)), ((0,), (1,), (2,), (1, 2), (0, 1)), ((0,), (1,), (2,), (0, 2), (0, 1, 2)), ((0,), (1,), (1, 2)), ((0,), (1,), (1, 2), (0, 2)), ((0,), (1,), (1, 2), (0, 1, 2)), ((0,), (1,), (1, 2), (0, 2), (0, 1, 2)), ((0,), (1,), (1, 2), (0, 1), (2,), (0, 2)), ((0,), (1,), (1, 2), (0, 1), (2,), (0, 1, 2)), ((0,), (1,), (1, 2), (0, 1), (0, 2), (0, 1, 2)), ((0,), (1,), (1, 2), (2,), (0, 2), (0, 1, 2)), ((0,), (1,), (2,), (0, 1), (0, 2), (0, 1, 2)), ((0,), (1, 2), (0, 1)), ((0,), (1, 2), (0, 1), (0, 2)), ((0,), (1, 2), (0, 1), (0, 1, 2)), ((0,), (1,

100%|██████████| 168/168 [24:51<00:00,  8.88s/it]


...

Theta min:  [((0,), (1,)), ((3,), (1,))]
non contradictory thetas :  [((0,), (1,), (2,), (0, 1), (0, 3)), ((0,), (1,), (2,), (0, 1), (3,)), ((0,), (1,), (2,), (0, 1), (0, 1, 3), (0, 3)), ((0,), (1,), (2,), (0, 1), (0, 1, 3), (3,)), ((0,), (1,), (2,), (0, 1), (1, 3), (0, 3)), ((0,), (1,), (2,), (0, 1), (1, 3), (3,)), ((0,), (1,), (2,), (1, 3), (3,)), ((0,), (1,), (2,), (1, 3), (0, 1, 3), (0, 3)), ((0,), (1,), (2,), (1, 3), (0, 1, 3), (3,)), ((0,), (1,), (2,), (0, 1, 3), (0, 3)), ((0,), (1,), (2,), (0, 1, 3), (3,)), ((0,), (1,), (2,), (0, 3), (0, 1), (1, 2)), ((0,), (1,), (2,), (0, 3), (0, 1), (3,)), ((0,), (1,), (2,), (0, 3), (0, 1, 3), (1, 2)), ((0,), (1,), (2,), (0, 3), (0, 1, 3), (3,)), ((0,), (1,), (2,), (0, 3), (1, 3), (3,)), ((0,), (1,), (2,), (3,), (0, 1), (1, 2)), ((0,), (1,), (2,), (3,), (0, 1, 3), (1, 2)), ((0,), (1,), (2,), (3,), (1, 3), (1, 2)), ((0, 3), (1,), (2,), (0, 1), (0, 1, 3)), ((0, 3), (1,), (2,), (0, 1), (1, 3)), ((0, 3), (1,), (2,), (0, 1), (0, 1, 3), (1, 2)), ((0

100%|██████████| 592/592 [1:24:49<00:00,  8.60s/it]


Theta min:  [((1,), (2,), (3,)), ((1,), (2,), (0,)), ((1,), (3,), (0,))]
non contradictory thetas :  []
non contradictory theta min :  []
Size of the intersection of theta preferences:  0
Size of the intersection of theta_min preferences: 39


100%|██████████| 546/546 [1:18:10<00:00,  8.59s/it]


Theta min:  [((1,), (0,))]
non contradictory thetas :  []
non contradictory theta min :  [((1,), (0,))]
Elements that are not contradictory in theta min are not in the not contradictory elements of theta
Size of the intersection of theta preferences:  0
Size of the intersection of theta_min preferences: 138
